# importSoil.ipynb

After downloading the necessairy files and configuring the paths below, this notebook loads and transforms the [Canadian soil data](https://sis.agr.gc.ca/cansis/nsdb/slc/v3.2/index.html) which is then uploaded as multiple tables into the database.

##### Required files:
- [SOIL_GEOM_FILE](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)
- [SOIL_COMP_FILE](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)
- [SOIL_RATING_FILE](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)
- [SOIL_SURRONDINGS_FILE](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)
- [MB_SOIL_NAMES_FILE](https://sis.agr.gc.ca/soildata/mb/soil_name_mb_v2r20130705.dbf)
- [MB_SOIL_LAYERS_FILE](https://sis.agr.gc.ca/soildata/mb/soil_layer_mb_v2r20130705.dbf)
- [AB_SOIL_NAMES_FILE](https://sis.agr.gc.ca/soildata/ab/soil_name_ab_v2r20140529.dbf)
- [AB_SOIL_LAYERS_FILE](https://sis.agr.gc.ca/soildata/ab/soil_layer_ab_v2r20140529.dbf)
- [SK_SOIL_NAMES_FILE](https://sis.agr.gc.ca/soildata/sk/soil_name_sk_v2r20130705.dbf)
- [SK_SOIL_LAYERS_FILE](https://sis.agr.gc.ca/soildata/sk/soil_layer_sk_v2r20130705.dbf)

##### Output tables:
- [soil_geometry](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions#soil_geometry)
- [soil_components](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions#soil_components)
- [soil_surronding_land](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions#soil_surronding_land)
- [soil_data](https://github.com/ChromaticPanic/CGC_Grain_Outcome_Predictions#soil_data)

In [ ]:
from dotenv import load_dotenv
import sqlalchemy as sq
import geopandas as gpd  # type: ignore
import pandas as pd
import os, sys

sys.path.append("../")
from Shared.DataService import DataService

In [ ]:
# The locations of the soil data source files used to populate the database
SOIL_GEOM_FILE = "data/ca_all_slc_v3r2.shp"
SOIL_COMP_FILE = "data/ca_all_slc_v3r2_cmp.dbf"
SOIL_RATING_FILE = "data/ca_all_slc_v3r2_crt.dbf"
SOIL_SURRONDINGS_FILE = "data/ca_all_slc_v3r2_lat.dbf"

MB_SOIL_NAMES_FILE = "data/soil_name_mb_v2r20130705.dbf"
MB_SOIL_LAYERS_FILE = "data/soil_layer_mb_v2r20130705.dbf"

AB_SOIL_NAMES_FILE = "data/soil_name_ab_v2r20140529.dbf"
AB_SOIL_LAYERS_FILE = "data/soil_layer_ab_v2r20140529.dbf"

SK_SOIL_NAMES_FILE = "data/soil_name_sk_v2r20130705.dbf"
SK_SOIL_LAYERS_FILE = "data/soil_layer_sk_v2r20130705.dbf"

SOIL_GEOM_TABLE = "soil_geometry"  # name of the table we are creating to hold the geometry/border data
SOIL_COMP_TABLE = "soil_components"  # name of the table we are creating to hold the soil component data
SOIL_SURRONDINGS_TABLE = "soil_surronding_land"  # name of the table we are creating the hold the surronding soil data
SOIL_DATA_TABLE = "soil_data"  # name of the table we are creating to hold the soil data


# Load the database connection environment variables located in the docker folder
load_dotenv("../docker/.env")
PG_DB = os.getenv("POSTGRES_DB")
PG_ADDR = os.getenv("POSTGRES_ADDR")
PG_PORT = os.getenv("POSTGRES_PORT")
PG_USER = os.getenv("POSTGRES_USER")
PG_PW = os.getenv("POSTGRES_PW")

In [ ]:
def main():
    if (
        PG_DB is None
        or PG_ADDR is None
        or PG_PORT is None
        or PG_USER is None
        or PG_PW is None
    ):
        raise ValueError("Environment variables not set")

    db = DataService(PG_DB, PG_ADDR, int(PG_PORT), PG_USER, PG_PW)
    conn = db.connect()

    loadSoilGeometries(conn)
    loadSoilComponents(conn)
    loadSoilSurrondings(conn)
    loadSoilData(conn)

    db.cleanup()

Purpose:  
Loads the soil geometry data downloaded from [here](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)

Psuedocode:
- [Load the geometry file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
- [Set the coordinate system](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.set_crs.html), *EPSG:4326* is the default and what we typically use in our day to day i.e GPS, *EPSG:3347* is the coordinate system we want
- [Change coordinates systems](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_crs.html) to EPSG:3347
- [Drop irrelevant columns](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
- [Rename columns to ease future database interactions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)
- [Upload the soil geometries](https://geopandas.org/en/stable/docs/reference/api/geopandas.GeoDataFrame.to_postgis.html)

Note: 
The inplace parameter makes it so that the dataframe itself is modified (as opposed to a copy). Therefore, the following two line are equivilant:
1. soil_data.rename(columns={"id": "soil_id"}, inplace=True)    
2. soil_data = soil_data.rename(columns={"id": "soil_id"})

In [ ]:
def loadSoilGeometries(conn: sq.engine.Connection):
    soilGeom = gpd.read_file(SOIL_GEOM_FILE, encoding="utf-8")

    soilGeom.set_crs(crs="EPSG:4326", allow_override=True, inplace=True)
    soilGeom.to_crs(crs="EPSG:3347", inplace=True)

    soilGeom.drop(columns=["ECO_ID"], inplace=True)

    soilGeom.rename(columns={"AREA": "area"}, inplace=True)
    soilGeom.rename(columns={"PERIMETER": "perimeter"}, inplace=True)
    soilGeom.rename(columns={"POLY_ID": "poly_id"}, inplace=True)

    soilGeom.to_postgis(SOIL_GEOM_TABLE, conn, index=False, if_exists="replace")

Purpose:  
Loads the soil components and soil ratings data downloaded from [here](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)

Psuedocode:
- [Load the components file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometry](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
- [Load the ratings file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometry](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
- [Merge both Dataframes](https://pandas.pydata.org/docs/reference/api/pandas.merge.html) on CMP_ID (unique component identifier)
- [Drop irrelevant columns](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
- [Rename columns to ease future database interactions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)
- [Upload the soil components](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)

Note: 
The inplace parameter makes it so that the dataframe itself is modified (as opposed to a copy). Therefore, the following two line are equivilant:
1. soil_data.rename(columns={"id": "soil_id"}, inplace=True)    
2. soil_data = soil_data.rename(columns={"id": "soil_id"})

In [ ]:
def loadSoilComponents(conn: sq.engine.Connection):
    soilComp = gpd.read_file(SOIL_COMP_FILE, encoding="utf-8")
    soilComp = pd.DataFrame(soilComp.drop(columns="geometry"))
    soilCompRating = gpd.read_file(SOIL_RATING_FILE, encoding="utf-8")
    soilCompRating = pd.DataFrame(soilCompRating.drop(columns="geometry"))

    # merge both the soil component and soil component rating data together
    soilComp = soilComp.merge(soilCompRating, on="CMP_ID")

    soilComp.drop(columns=["CMP_ID", "RESTR_TYPE"], inplace=True)

    soilComp.rename(columns={"POLY_ID": "poly_id"}, inplace=True)
    soilComp.rename(columns={"CMP": "cmp"}, inplace=True)
    soilComp.rename(columns={"PERCENT": "percent"}, inplace=True)
    soilComp.rename(columns={"SLOPE": "slope"}, inplace=True)
    soilComp.rename(columns={"STONE": "stone"}, inplace=True)
    soilComp.rename(columns={"LOCSF": "surface_area"}, inplace=True)
    soilComp.rename(columns={"PROVINCE": "province"}, inplace=True)
    soilComp.rename(columns={"SOIL_CODE": "soil_code"}, inplace=True)
    soilComp.rename(columns={"MODIFIER": "modifier"}, inplace=True)
    soilComp.rename(columns={"PROFILE": "profile"}, inplace=True)
    soilComp.rename(columns={"SOIL_ID": "soil_id"}, inplace=True)
    soilComp.rename(columns={"CFRAG1": "coarse_frag_1"}, inplace=True)
    soilComp.rename(columns={"CFRAG2": "coarse_frag_2"}, inplace=True)
    soilComp.rename(columns={"DEPTH": "depth"}, inplace=True)
    soilComp.rename(columns={"AWHC": "water_holding_cap"}, inplace=True)

    soilComp.to_sql(SOIL_COMP_TABLE, conn, index=False, if_exists="replace")

Purpose:  
Loads the surronding soil data downloaded from [here](https://sis.agr.gc.ca/nsdb/ca/cac003/cac003.20110308.v3.2/ca_all_slc_v3r2.zip)

Psuedocode:
- [Load the surronding data file](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometry](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
- [Rename columns to ease future database interactions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)
- [Upload the soil components](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)

Note: 
The inplace parameter makes it so that the dataframe itself is modified (as opposed to a copy). Therefore, the following two line are equivilant:
1. soil_data.rename(columns={"id": "soil_id"}, inplace=True)    
2. soil_data = soil_data.rename(columns={"id": "soil_id"})

In [ ]:
def loadSoilSurrondings(conn: sq.engine.Connection):
    soilSurrondings = gpd.read_file(SOIL_SURRONDINGS_FILE, encoding="utf-8")
    soilSurrondings = pd.DataFrame(soilSurrondings.drop(columns="geometry"))

    soilSurrondings.rename(columns={"POLY_ID": "poly_id"}, inplace=True)
    soilSurrondings.rename(columns={"LAND_AREA": "land_area"}, inplace=True)
    soilSurrondings.rename(columns={"WATER_AREA": "water_area"}, inplace=True)
    soilSurrondings.rename(columns={"FRESH_AREA": "fresh_area"}, inplace=True)
    soilSurrondings.rename(columns={"OCEAN_AREA": "ocean_area"}, inplace=True)
    soilSurrondings.rename(columns={"TOTAL_AREA": "total_area"}, inplace=True)

    soilSurrondings.to_sql(
        SOIL_SURRONDINGS_TABLE, conn, index=False, if_exists="replace"
    )

Purpose:  
Loads soil name and layer data downloaded from:
- [Manitoba names](https://sis.agr.gc.ca/soildata/mb/soil_name_mb_v2r20130705.dbf)
- [Manitoba layers](https://sis.agr.gc.ca/soildata/mb/soil_layer_mb_v2r20130705.dbf)
- [Alberta names](https://sis.agr.gc.ca/soildata/ab/soil_name_ab_v2r20140529.dbf)
- [Alberta layers](https://sis.agr.gc.ca/soildata/ab/soil_layer_ab_v2r20140529.dbf)
- [Saskatchewan names](https://sis.agr.gc.ca/soildata/sk/soil_name_sk_v2r20130705.dbf)
- [Saskatchewan layers](https://sis.agr.gc.ca/soildata/sk/soil_layer_sk_v2r20130705.dbf)

Psuedocode:
- [Load both Manitoba data files](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometries](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
    - [Merge both Dataframes](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
        - *how=inner* (default behavior) specifies that all non intersecting data should be removed
        - *on=the intersection of the columns in both DataFrames* (default behavior)
- [Load both Alberta data files](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometries](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
    - [Merge both Dataframes](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
        - *how=inner* (default behavior) specifies that all non intersecting data should be removed
        - *on=the intersection of the columns in both DataFrames* (default behavior)
- [Load both Saskatchewan data files](https://geopandas.org/en/stable/docs/reference/api/geopandas.read_file.html)
    - [Drop geometries](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html) and convert to a DataFrame
    - [Merge both Dataframes](https://pandas.pydata.org/docs/reference/api/pandas.merge.html)
        - *how=inner* (default behavior) specifies that all non intersecting data should be removed
        - *on=the intersection of the columns in both DataFrames* (default behavior)
- [Drop irrelevant columns](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
- [Rename columns to ease future database interactions](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rename.html)
- [Upload the soil components](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html)

Note: 
The inplace parameter makes it so that the dataframe itself is modified (as opposed to a copy). Therefore, the following two line are equivilant:
1. soil_data.rename(columns={"id": "soil_id"}, inplace=True)    
2. soil_data = soil_data.rename(columns={"id": "soil_id"})

In [ ]:
def loadSoilData(conn: sq.engine.Connection):
    # load the data for the soils present in Manitoba
    mbSoilName = gpd.read_file(MB_SOIL_NAMES_FILE, encoding="utf-8")
    mbSoilName = pd.DataFrame(mbSoilName.drop(columns="geometry"))
    mbSoilLayer = gpd.read_file(MB_SOIL_LAYERS_FILE, encoding="utf-8")
    mbSoilLayer = pd.DataFrame(mbSoilLayer.drop(columns="geometry"))
    mbSoil = mbSoilName.merge(mbSoilLayer)

    # load the data for the soils present in Alberta
    abSoilName = gpd.read_file(AB_SOIL_NAMES_FILE, encoding="utf-8")
    abSoilName = pd.DataFrame(abSoilName.drop(columns="geometry"))
    abSoilLayer = gpd.read_file(AB_SOIL_LAYERS_FILE, encoding="utf-8")
    abSoilLayer = pd.DataFrame(abSoilLayer.drop(columns="geometry"))
    abSoil = abSoilName.merge(abSoilLayer)

    # load the data for the soils present in Saskatchewan
    skSoilName = gpd.read_file(SK_SOIL_NAMES_FILE, encoding="utf-8")
    skSoilName = pd.DataFrame(skSoilName.drop(columns="geometry"))
    skSoilLayer = gpd.read_file(SK_SOIL_LAYERS_FILE, encoding="utf-8")
    skSoilLayer = pd.DataFrame(skSoilLayer.drop(columns="geometry"))
    skSoil = skSoilName.merge(skSoilLayer)
    soilData = pd.concat([mbSoil, abSoil, skSoil])

    soilData.drop(
        columns=[
            "PROFILE",
            "ORDER2",
            "G_GROUP2",
            "S_GROUP2",
            "ORDER3",
            "G_GROUP3",
            "S_GROUP3",
        ],
        inplace=True,
    )

    soilData.rename(columns={"SOIL_ID": "id"}, inplace=True)
    soilData.rename(columns={"PROVINCE": "province"}, inplace=True)
    soilData.rename(columns={"SOIL_CODE": "code"}, inplace=True)
    soilData.rename(columns={"MODIFIER": "modifier"}, inplace=True)
    soilData.rename(columns={"SOILNAME": "name"}, inplace=True)
    soilData.rename(columns={"KIND": "kind"}, inplace=True)
    soilData.rename(columns={"WATERTBL": "water_table"}, inplace=True)
    soilData.rename(columns={"ROOTRESTRI": "root_restrict"}, inplace=True)
    soilData.rename(columns={"RESTR_TYPE": "restr_type"}, inplace=True)
    soilData.rename(columns={"DRAINAGE": "drainage"}, inplace=True)
    soilData.rename(columns={"PMTEX1": "parent_material_texture_1"}, inplace=True)
    soilData.rename(columns={"PMTEX2": "parent_material_texture_2"}, inplace=True)
    soilData.rename(columns={"PMTEX3": "parent_material_texture_3"}, inplace=True)
    soilData.rename(columns={"PMCHEM1": "parent_material_chemical_1"}, inplace=True)
    soilData.rename(columns={"PMCHEM2": "parent_material_chemical_2"}, inplace=True)
    soilData.rename(columns={"PMCHEM3": "parent_material_chemical_3"}, inplace=True)
    soilData.rename(columns={"MDEP1": "mode_of_depo_1"}, inplace=True)
    soilData.rename(columns={"MDEP2": "mode_of_depo_2"}, inplace=True)
    soilData.rename(columns={"MDEP3": "mode_of_depo_3"}, inplace=True)
    soilData.rename(columns={"LAYER_NO": "layer_no"}, inplace=True)
    soilData.rename(columns={"UDEPTH": "u_depth"}, inplace=True)
    soilData.rename(columns={"LDEPTH": "l_depth"}, inplace=True)
    soilData.rename(columns={"HZN_LIT": "hzn_lit"}, inplace=True)
    soilData.rename(columns={"HZN_MAS": "hzn_mas"}, inplace=True)
    soilData.rename(columns={"HZN_SUF": "hzn_suf"}, inplace=True)
    soilData.rename(columns={"HZN_MOD": "hzn_mod"}, inplace=True)
    soilData.rename(columns={"COFRAG": "percnt_coarse_frag"}, inplace=True)
    soilData.rename(columns={"DOMSAND": "sand_texture"}, inplace=True)
    soilData.rename(columns={"VFSAND": "percnt_v_fine_sand"}, inplace=True)
    soilData.rename(columns={"TSAND": "total_sand"}, inplace=True)
    soilData.rename(columns={"TSILT": "total_silt"}, inplace=True)
    soilData.rename(columns={"TCLAY": "total_clay"}, inplace=True)
    soilData.rename(columns={"ORGCARB": "percnt_carbon"}, inplace=True)
    soilData.rename(columns={"PHCA": "calcium_ph"}, inplace=True)
    soilData.rename(columns={"PH2": "proj_ph"}, inplace=True)
    soilData.rename(columns={"BASES": "percnt_base_sat"}, inplace=True)
    soilData.rename(columns={"CEC": "cec"}, inplace=True)
    soilData.rename(columns={"KSAT": "ksat"}, inplace=True)
    soilData.rename(columns={"KP0": "water_reten_0"}, inplace=True)
    soilData.rename(columns={"KP10": "water_reten_10"}, inplace=True)
    soilData.rename(columns={"KP33": "water_reten_33"}, inplace=True)
    soilData.rename(columns={"KP1500": "water_reten_1500"}, inplace=True)
    soilData.rename(columns={"BD": "bulk_density"}, inplace=True)
    soilData.rename(columns={"EC": "elec_cond"}, inplace=True)
    soilData.rename(columns={"CACO3": "calc_equiv"}, inplace=True)
    soilData.rename(columns={"VONPOST": "decomp_class"}, inplace=True)
    soilData.rename(columns={"WOOD": "percnt_wood"}, inplace=True)

    soilData.to_sql(
        SOIL_DATA_TABLE, conn, schema="public", if_exists="append", index=False
    )

In [ ]:
if __name__ == "__main__":
    main()